In [1]:
%run ../src/parsing/preset.py

In [2]:
res = make_subj_pred(7, 'w', "../data/feature_data.gzip", "../models/XGBoost", "Южно-Якутский энергорайон")

In [3]:
res.head(5)

,time_dummie,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday,...,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область,lag_hour,lag_day,lag_week,lag_month,lag_year,actual_consumption
5,114146,2024,5,9,0,4,130,19,3,1,...,0,0,1,0,378.0,354.0,396.0,417.0,381.0,359.893890
80,114147,2024,5,9,1,4,130,19,3,1,...,0,0,1,0,382.0,345.0,388.0,414.0,381.0,356.791443
155,114148,2024,5,9,2,4,130,19,3,1,...,0,0,1,0,374.0,346.0,398.0,405.0,375.0,355.400909
230,114149,2024,5,9,3,4,130,19,3,1,...,0,0,1,0,380.0,343.0,392.0,402.0,381.0,361.630371
305,114150,2024,5,9,4,4,130,19,3,1,...,0,0,1,0,388.0,344.0,401.0,396.0,382.0,370.991974


In [4]:
res.tail(5)

,time_dummie,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,holiday,...,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область,lag_hour,lag_day,lag_week,lag_month,lag_year,actual_consumption
87898,115798,2024,7,16,21,2,198,29,3,0,...,0,0,1,0,352.0,354.0,327.0,351.0,388.0,351.195068
87973,115799,2024,7,16,22,2,198,29,3,0,...,0,0,1,0,345.0,353.0,328.0,350.0,402.0,335.965027
88048,115800,2024,7,16,23,2,198,29,3,0,...,0,0,1,0,348.0,349.0,307.0,325.0,398.0,336.033081
88196,1,2024,7,17,0,3,199,29,3,0,...,0,0,1,0,358.0,344.0,337.0,343.0,386.0,336.315643
88197,115801,2024,7,17,0,3,199,29,3,0,...,0,0,1,0,357.0,357.0,306.0,338.0,392.0,341.546814
